# Quick Start 

**Home GitHub Repository:** [LEANN on GitHub](https://github.com/yichuan-w/LEANN)

**Important for Colab users:** Set your runtime type to T4 GPU for optimal performance. Go to Runtime → Change runtime type → Hardware accelerator → T4 GPU.

In [ ]:
# install this if you are using colab
! uv pip install leann-core leann-backend-hnsw --no-deps
! uv pip install leann --no-deps
# For Colab environment, we need to set some environment variables
import os

os.environ["LEANN_LOG_LEVEL"] = "INFO"  # Enable more detailed logging

In [ ]:
from pathlib import Path

INDEX_DIR = Path("./").resolve()
INDEX_PATH = str(INDEX_DIR / "demo.leann")

## Build the index

In [ ]:
from leann.api import LeannBuilder

builder = LeannBuilder(backend_name="hnsw")
builder.add_text("C# is a powerful programming language and it is good at game development")
builder.add_text(
    "Python is a powerful programming language and it is good at machine learning tasks"
)
builder.add_text("Machine learning transforms industries")
builder.add_text("Neural networks process complex data")
builder.add_text("Leann is a great storage saving engine for RAG on your MacBook")
builder.build_index(INDEX_PATH)

## Search with real-time embeddings

In [ ]:
from leann.api import LeannSearcher

searcher = LeannSearcher(INDEX_PATH)
results = searcher.search("programming languages", top_k=2)
results

## Chat with LEANN using retrieved results

In [ ]:
from leann.api import LeannChat

llm_config = {
    "type": "hf",
    "model": "Qwen/Qwen3-0.6B",
}

chat = LeannChat(index_path=INDEX_PATH, llm_config=llm_config)
response = chat.ask(
    "Compare the two retrieved programming languages and tell me their advantages.",
    top_k=2,
    llm_kwargs={"max_tokens": 128},
)
response